In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_json('./ind-kabkot-rev.json')

In [3]:
provinsi = df.groupby(['provinsi']).size().reset_index().provinsi.tolist()

In [4]:
provinsi

['Aceh',
 'Bali',
 'Banten',
 'Bengkulu',
 'Gorontalo',
 'Jakarta',
 'Jambi',
 'Jawa Barat',
 'Jawa Tengah',
 'Jawa Timur',
 'Kalimantan Barat',
 'Kalimantan Selatan',
 'Kalimantan Tengah',
 'Kalimantan Timur',
 'Kalimantan Utara',
 'Kepulauan Bangka Belitung',
 'Kepulauan Riau',
 'Lampung',
 'Maluku',
 'Maluku Utara',
 'Nusa Tenggara Barat',
 'Nusa Tenggara Timur',
 'Papua',
 'Papua Barat',
 'Riau',
 'Sulawesi Barat',
 'Sulawesi Selatan',
 'Sulawesi Tengah',
 'Sulawesi Tenggara',
 'Sulawesi Utara',
 'Sumatera Barat',
 'Sumatera Selatan',
 'Sumatera Utara',
 'Yogyakarta']

In [5]:
# make folder

In [5]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [6]:
for f in provinsi:
    folder_name = '../'+ f
    createFolder(folder_name)

In [8]:
# writing makefile

In [7]:
makefile_text = """map: filter-map-by-province build-geo build-topo simplify-topo collect-bahasa-map clean
.PHONY: clean

filter-map-by-province:
\tndjson-filter 'd.properties.provinsi === "{prov}"' < ../data/ind-map.ndjson > {filename}.ndjson

build-geo:
\tndjson-reduce 'p.features.push(d), p' '{{type: "FeatureCollection", features: []}}' < {filename}.ndjson > {filename}.json

build-topo:
\ttopojson {filename}.json > {filename}-topo.json -p

simplify-topo:
\tmapshaper {filename}-topo.json -simplify 3% -o format=topojson {filename}-simplified-topo.json

collect-bahasa-map:
\tcp {filename}-simplified-topo.json ../All

clean:
\trm -rf {filename}.ndjson {filename}.json {filename}-topo.json
"""

In [8]:
for p in provinsi:
    fileloc = '../'+ p + '/Makefile'
    filename = p.lower().replace(' ', '-')
    
    makefile = makefile_text.format(prov=p, filename=filename)
    #print(makefile)
    with open(fileloc, 'w') as outfile:
        outfile.write(makefile)

In [19]:
## Create readme table

readme_template = """
| Provinsi | Maps |
| -------- | ---- |
"""

for p in provinsi:
    filename = p.lower().replace(' ', '-')
    folder = p.replace(' ', '&#32');
    
    readme_template += """\n| {title} | [File](kabupaten-kota/{folder}/{filename}-simplified-topo.json) |""".format(title=p, folder=folder, filename=filename)
    
with open('readme.md', 'w') as outfile:
    outfile.write(readme_template)